#Проект
Задача: создать прогностическую модель рисков беременных.   
--Age: Возраст в годах, когда женщина стала беременна (Age in years when a woman is pregnant).   
--SystolicBP: Верхнее значение артериального давления в мм рт.ст., еще один важный признак во время беременности (value of Blood Pressure in mmHg, another significant attribute during pregnancy).   
--DiastolicBP: Нижнее значение артериального давления в мм рт.ст., еще один важный признак во время беременности (Lower value of Blood Pressure in mmHg, another significant attribute during pregnancy).   
--BS: Уровень глюкозы в крови выражается в молярной концентрации, ммоль/л (Blood glucose levels is in terms of a molar concentration, mmol/L).   
--HeartRate: Нормальная частота сердечных сокращений в состоянии покоя в ударах в минуту (A normal resting heart rate in beats per minute).      
--Risk Level: Прогнозируемый уровень интенсивности риска во время беременности с учетом предыдущего признака (Predicted Risk Intensity Level during pregnancy considering the previous attribute).   

1.   Изучить рабочую задачу,изучить данные в файлах - проверить типы, есть ли пропуски, надо ли преобразовывать данные. Сделать первичный вывод
2.   Подготовка данных -провести предобработку, выделить целевой признак, подготовить фичи к обучению моделей.
3. Выбор моделей, подбор гиперпараметров, проверка результатов, анализ лучшей модели.
4.  Итоговый вывод.


###  1.Изучить рабочую задачу,изучить данные в файлах - проверить типы, есть ли пропуски, надо ли преобразовывать данные. Сделать первичный вывод

In [ ]:
import pandas as pd

In [ ]:
pregn_risk = pd.read_csv('/content/Maternal Health Risk Data Set.csv')

In [ ]:
pregn_risk.head(15)

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,130,80,15.00,98.0,86,high risk
1,35,140,90,13.00,98.0,70,high risk
2,29,90,70,8.00,100.0,80,high risk
3,30,140,85,7.00,98.0,70,high risk
4,35,120,60,6.10,98.0,76,low risk
5,23,140,80,7.01,98.0,70,high risk
6,23,130,70,7.01,98.0,78,mid risk
7,35,85,60,11.00,102.0,86,high risk
8,32,120,90,6.90,98.0,70,mid risk
9,42,130,80,18.00,98.0,70,high risk


In [ ]:
pregn_risk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1014 entries, 0 to 1013
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Age          1014 non-null   int64  
 1   SystolicBP   1014 non-null   int64  
 2   DiastolicBP  1014 non-null   int64  
 3   BS           1014 non-null   float64
 4   BodyTemp     1014 non-null   float64
 5   HeartRate    1014 non-null   int64  
 6   RiskLevel    1014 non-null   object 
dtypes: float64(2), int64(4), object(1)
memory usage: 55.6+ KB


In [ ]:
pregn_risk.isna().sum()

Age            0
SystolicBP     0
DiastolicBP    0
BS             0
BodyTemp       0
HeartRate      0
RiskLevel      0
dtype: int64

посмотрим скрытые пропуски подствляя название каждого столбца

In [ ]:
pregn_risk.loc[pregn_risk['HeartRate']==' ']

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel


In [ ]:
pregn_risk.duplicated().sum()

562

In [ ]:
pregn_risk[pregn_risk.duplicated(keep=False)][pregn_risk[pregn_risk.duplicated(keep=False)].duplicated(subset=pregn_risk.columns.difference(['RiskLevel']), keep=False)].sort_values(by=pregn_risk.columns.tolist())

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
670,10,100,50,6.0,99.0,70,mid risk
849,10,100,50,6.0,99.0,70,mid risk
552,12,90,60,7.5,102.0,60,low risk
940,12,90,60,7.5,102.0,60,low risk
543,12,90,60,7.5,102.0,66,low risk
...,...,...,...,...,...,...,...
553,60,120,85,15.0,98.0,60,mid risk
772,60,120,85,15.0,98.0,60,mid risk
818,60,120,85,15.0,98.0,60,mid risk
114,63,140,90,15.0,98.0,90,high risk


посмотрели 18 дубликатов, которые отличаются только по столбцу RiskLevel

In [ ]:
len(pregn_risk[pregn_risk.duplicated(subset=['Age', 'SystolicBP', 'DiastolicBP', 'BS', 'BodyTemp', 'HeartRate'], keep=False)
& ~pregn_risk.duplicated(subset=['Age', 'SystolicBP', 'DiastolicBP', 'BS', 'BodyTemp', 'HeartRate', 'RiskLevel'], keep=False)])

18

In [ ]:
pregn_risk[pregn_risk.duplicated(subset=['Age', 'SystolicBP', 'DiastolicBP', 'BS', 'BodyTemp', 'HeartRate'], keep=False)
& ~pregn_risk.duplicated(subset=['Age', 'SystolicBP', 'DiastolicBP', 'BS', 'BodyTemp', 'HeartRate', 'RiskLevel'], keep=False)]

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
251,12,95,60,6.9,98.0,65,low risk
252,28,120,90,6.9,98.0,70,low risk
253,40,120,90,6.9,98.0,80,low risk
254,55,110,85,6.9,98.0,88,low risk
255,25,100,90,6.9,98.0,76,low risk
256,35,120,80,6.9,98.0,80,low risk
257,21,90,65,6.9,98.0,76,low risk
258,18,90,60,6.9,98.0,70,low risk
259,21,120,80,6.9,98.0,76,low risk
260,16,100,70,6.9,98.0,80,low risk


In [ ]:
duplicates = pregn_risk[pregn_risk.duplicated(subset=['Age', 'SystolicBP', 'DiastolicBP', 'BS', 'BodyTemp', 'HeartRate'], keep=False)
                        & ~pregn_risk.duplicated(subset=['Age', 'SystolicBP', 'DiastolicBP', 'BS', 'BodyTemp', 'HeartRate', 'RiskLevel'], keep=False)]

unique_risk = pregn_risk[pregn_risk.duplicated(subset=['Age', 'SystolicBP', 'DiastolicBP', 'BS', 'BodyTemp', 'HeartRate'], keep=False)]

output_rows = []

for index, row in duplicates.iterrows():
    copy_row = unique_risk[(unique_risk['Age'] == row['Age']) & (unique_risk['SystolicBP'] == row['SystolicBP']) & (unique_risk['DiastolicBP'] == row['DiastolicBP']) & (unique_risk['BS'] == row['BS']) & (unique_risk['BodyTemp'] == row['BodyTemp']) & (unique_risk['HeartRate'] == row['HeartRate'])]
    if not copy_row.empty:
        output_rows.append(row)
        output_rows.append(copy_row.iloc[0])

output_df = pd.DataFrame(output_rows)
output_df

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
251,12,95,60,6.9,98.0,65,low risk
82,12,95,60,6.9,98.0,65,mid risk
252,28,120,90,6.9,98.0,70,low risk
83,28,120,90,6.9,98.0,70,mid risk
253,40,120,90,6.9,98.0,80,low risk
253,40,120,90,6.9,98.0,80,low risk
254,55,110,85,6.9,98.0,88,low risk
254,55,110,85,6.9,98.0,88,low risk
255,25,100,90,6.9,98.0,76,low risk
255,25,100,90,6.9,98.0,76,low risk


In [ ]:
pregn_risk[(pregn_risk['Age']>=40)]

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
9,42,130,80,18.0,98.0,70,high risk
14,48,120,80,11.0,98.0,88,mid risk
16,50,140,90,15.0,98.0,90,high risk
20,40,140,100,18.0,98.0,90,high risk
21,50,140,80,6.7,98.0,70,mid risk
...,...,...,...,...,...,...,...
1003,50,130,100,16.0,98.0,76,high risk
1007,40,120,75,7.7,98.0,70,high risk
1008,48,120,80,11.0,98.0,88,high risk
1010,55,120,90,18.0,98.0,60,high risk


42 после 60

In [ ]:
pregn_risk[(pregn_risk['Age']>=55)]

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
36,60,120,80,6.1,98.0,75,low risk
37,55,100,65,6.1,98.0,66,low risk
54,60,90,65,7.0,98.0,77,low risk
77,55,120,90,12.0,98.0,70,mid risk
91,60,120,85,15.0,98.0,60,mid risk
...,...,...,...,...,...,...,...
968,55,140,95,19.0,98.0,77,high risk
982,55,140,80,7.2,101.0,76,high risk
1001,60,120,85,15.0,98.0,60,high risk
1002,55,140,95,19.0,98.0,77,high risk


In [ ]:
pregn_risk[(pregn_risk['Age']>=40) &(pregn_risk['RiskLevel']== 'low risk')]

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
28,49,120,90,7.2,98.0,77,low risk
36,60,120,80,6.1,98.0,75,low risk
37,55,100,65,6.1,98.0,66,low risk
38,45,120,95,6.1,98.0,66,low risk
46,42,120,80,7.5,98.0,70,low risk
...,...,...,...,...,...,...,...
941,60,120,80,7.5,98.0,75,low risk
942,55,100,65,7.5,98.0,66,low risk
943,45,120,95,7.5,98.0,66,low risk
949,59,120,80,7.5,98.0,70,low risk


распределение данных в таргете нормальное

In [ ]:
pregn_risk['RiskLevel'].value_counts()

low risk     406
mid risk     336
high risk    272
Name: RiskLevel, dtype: int64

 Выводы по данным:   
 Был рассмотрен датасет с данными, в нем 7 столбцов, хотя предоставлены была информация только о шести.
Дополнительно найден столбец BodyTemp, который судя по названию содержит информацию о температуре
тела пациентов, судя по данным в этом столбце температура указана по Фаренгейту. Однако будет лучше окончательно
уточнить описание столбца у заказчика. В датасете 1014 строк, сразу обращает внимание такая небольшая
представленность данных. Дальнейшее их рассмотрение выявило, что хотя пропусков в том числе неявных не имеется, но
зато дубликаты составляют более половины датасета 562. Т.к. у нас отсутствуют ID пациентов, мы не можем сказать
отдельные это клиенты с одинаково заполненными данными или какой-то сбой и повторение данных на одного пациента,
необходимо уточнить у заказчика, т.к. это имеет существенное значение для исследования. Дальнейшее рассмотрение данных выявило, что
что есть дубликаты, которые содержат идентичные данные, а RiskLevel(уровень риска) разный, причем даже противоположный
low risk(низкий) и high risk(высокий). Это вызывает сомнения в адекватности разметки данных. Кроме того данных по возрасту
старше 40 почти 1/4 датасета, немалому количеству поставлен low risk(низкий) столбца RiskLevel(уровень риска), такая разметка
вызывает вопросы, особенно для группы старше 55. В связи со множеством вопросов принято решение передать их для прояснения заказчику,
также возможно будет предоставлен ID пациентов для решения вопроса по дубликатам, а также больший объем самих данных.